In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os

dataset_path = '/content/drive/MyDrive/Colab Notebooks/datasets/RockPaperScissorDataset'

train_dir = '/content/drive/MyDrive/Colab Notebooks/datasets/RockPaperScissorDataset/train'
validation_dir = '/content/drive/MyDrive/Colab Notebooks/datasets/RockPaperScissorDataset/validation'

os.makedirs(train_dir, exist_ok=True)
os.makedirs(validation_dir, exist_ok=True)

In [ ]:
import shutil
from sklearn.model_selection import train_test_split

split_ratio = 0.8

for class_name in ['rock', 'paper', 'scissors']:
    class_path = os.path.join(dataset_path, class_name)

    images = os.listdir(class_path)

    train_images, val_images = train_test_split(images, train_size=split_ratio, random_state=42)

    os.makedirs(os.path.join(train_dir, class_name), exist_ok=True)
    os.makedirs(os.path.join(validation_dir, class_name), exist_ok=True)

    for image in train_images:
        src = os.path.join(class_path, image)
        dst = os.path.join(train_dir, class_name, image)
        shutil.copy(src, dst)

    for image in val_images:
        src = os.path.join(class_path, image)
        dst = os.path.join(validation_dir, class_name, image)
        shutil.copy(src, dst)

print("Dataset split into train and validation sets!")

Dataset split into train and validation sets!


In [ ]:
for class_name in ['rock', 'paper', 'scissors']:
    train_class_path = os.path.join(train_dir, class_name)
    val_class_path = os.path.join(validation_dir, class_name)

    print(f"Train {class_name}: {len(os.listdir(train_class_path))} images")
    print(f"Validation {class_name}: {len(os.listdir(val_class_path))} images")

Train rock: 580 images
Validation rock: 146 images
Train paper: 569 images
Validation paper: 143 images
Train scissors: 600 images
Validation scissors: 150 images


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

Found 1749 images belonging to 3 classes.
Found 439 images belonging to 3 classes.


In [ ]:
ase_model = MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet'
)

base_model.trainable = False
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224 (Functional)    │ (None, 7, 7, 1280)          │       2,257,984 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 1280)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 512)                 │         655,872 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 3)                   │           1,539 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,915,395 (11.12 MB)

 Trainable params: 657,411 (2.51 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=10
)
base_model.trainable = True

for layer in base_model.layers[:-10]:
    layer.trainable = False

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history_fine = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=5
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 100s 2s/step - accuracy: 0.8257 - loss: 0.4147 - val_accuracy: 0.9880 - val_loss: 0.0351
Epoch 2/10
 1/54 ━━━━━━━━━━━━━━━━━━━━ 48s 918ms/step - accuracy: 1.0000 - loss: 0.0368

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


54/54 ━━━━━━━━━━━━━━━━━━━━ 15s 273ms/step - accuracy: 1.0000 - loss: 0.0368 - val_accuracy: 0.9880 - val_loss: 0.0366
Epoch 3/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 143s 2s/step - accuracy: 0.9765 - loss: 0.0636 - val_accuracy: 0.9928 - val_loss: 0.0324
Epoch 4/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 16s 277ms/step - accuracy: 1.0000 - loss: 0.0132 - val_accuracy: 0.9928 - val_loss: 0.0323
Epoch 5/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 93s 2s/step - accuracy: 0.9851 - loss: 0.0443 - val_accuracy: 0.9928 - val_loss: 0.0332
Epoch 6/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 21s 378ms/step - accuracy: 1.0000 - loss: 0.0211 - val_accuracy: 0.9952 - val_loss: 0.0293
Epoch 7/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 140s 3s/step - accuracy: 0.9752 - loss: 0.0740 - val_accuracy: 0.9904 - val_loss: 0.0374
Epoch 8/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 20s 368ms/step - accuracy: 1.0000 - loss: 0.0267 - val_accuracy: 0.9880 - val_loss: 0.0392
Epoch 9/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 125s 2s/step - accuracy: 0.9876 - loss: 0.0343 - val_accuracy: 0.9976 - val_l

In [ ]:
loss, accuracy = model.evaluate(validation_generator)
print(f'Validation Loss: {loss}')
print(f'Validation Accuracy: {accuracy}')
model.save('/content/drive/MyDrive/Colab Notebooks/datasets/RockPaperScissorDataset/rps_mobilenetv2.h5')

14/14 ━━━━━━━━━━━━━━━━━━━━ 16s 1s/step - accuracy: 0.9903 - loss: 0.0283


Validation Loss: 0.029333194717764854
Validation Accuracy: 0.9931662678718567
